# Emerging Sales

In [55]:
import os

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

#from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [56]:
df1 = pd.read_csv("google_search_data.csv")
df2 = pd.read_csv("product_manufacturer_list.csv")
df3 = pd.read_csv("sales_data.csv")
df4 = pd.read_csv("social_media_data.csv")
df5 = pd.read_csv("Theme_list.csv")
df6= pd.read_csv("Theme_product_list.csv")

In [57]:
df1.columns = df1.columns.str.lower()

In [58]:
df2.columns = df2.columns.str.lower()

In [59]:
df3.columns = df3.columns.str.lower()

In [60]:
df4.columns = df4.columns.str.lower()

In [61]:
df5.columns = df5.columns.str.lower()

In [62]:
df6.columns = df6.columns.str.lower()

In [63]:
df1

,date,platform,searchvolume,claim_id,week_number,year_new
0,05-01-2014,google,349,916,1,2014
1,06-01-2014,google,349,916,2,2014
2,07-01-2014,google,697,916,2,2014
3,10-01-2014,google,349,916,2,2014
4,20-01-2014,google,697,916,4,2014
...,...,...,...,...,...,...
181560,05-09-2019,walmart,125,980,36,2019
181561,22-09-2019,walmart,84,980,38,2019
181562,26-09-2019,walmart,42,980,39,2019
181563,15-07-2019,walmart,42,622,29,2019


In [64]:
df2

,product_id,vendor,unnamed: 2,unnamed: 3,unnamed: 4,unnamed: 5,unnamed: 6
0,1,Others,NaN,NaN,NaN,NaN,NaN
1,2,Others,NaN,NaN,NaN,NaN,NaN
2,3,Others,NaN,NaN,NaN,NaN,NaN
3,4,Others,NaN,NaN,NaN,NaN,NaN
4,5,Others,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
67170,67171,Private Label,NaN,NaN,NaN,NaN,NaN
67171,67172,Private Label,NaN,NaN,NaN,NaN,NaN
67172,67173,Private Label,NaN,NaN,NaN,NaN,NaN
67173,67174,Private Label,NaN,NaN,NaN,NaN,NaN


In [65]:
df3

,system_calendar_key_n,product_id,sales_dollars_value,sales_units_value,sales_lbs_value
0,20160109,1,13927.0,934,18680
1,20160109,3,10289.0,1592,28646
2,20160109,4,357.0,22,440
3,20160109,6,23113.0,2027,81088
4,20160109,7,23177.0,3231,58164
...,...,...,...,...,...
4526177,20181027,47536,8.0,2,3
4526178,20181027,47539,391.0,39,68
4526179,20181027,47543,105.0,59,48
4526180,20181027,47544,3720.0,1246,4361


In [66]:
df4

,theme id,published_date,total_post
0,148.0,10-01-2015,76
1,148.0,10-10-2015,31
2,148.0,10-11-2015,65
3,148.0,10-12-2015,88
4,148.0,10/13/2015,85
...,...,...,...
533385,876.0,09-05-2019,4658
533386,876.0,09-06-2019,3731
533387,876.0,09-07-2019,2336
533388,876.0,09-08-2019,1374


In [67]:
df5

,claim_id,claim name
0,0,No Claim
1,8,low carb
2,15,beans
3,16,cocoa
4,26,vanilla
...,...,...
203,508,cola
204,769,shortbread
205,949,passion fruit
206,521,blood orange


In [68]:
df6

,product_id,claim_id
0,26,8
1,29,8
2,48,81
3,50,81
4,74,227
...,...,...
91480,8158,0
91481,45183,0
91482,25690,0
91483,46085,0


In [69]:
df10 = pd.merge(df1,df5, on = 'claim_id', how = 'inner')

In [70]:
df10

,date,platform,searchvolume,claim_id,week_number,year_new,claim name
0,05-01-2014,google,349,916,1,2014,gluten free
1,06-01-2014,google,349,916,2,2014,gluten free
2,07-01-2014,google,697,916,2,2014,gluten free
3,10-01-2014,google,349,916,2,2014,gluten free
4,20-01-2014,google,697,916,4,2014,gluten free
...,...,...,...,...,...,...,...
181560,24-07-2019,google,42,843,30,2019,snickerdoodle
181561,25-09-2019,google,84,843,39,2019,snickerdoodle
181562,04-06-2018,amazon,135,606,23,2018,hemp seeds
181563,31-01-2019,amazon,39,433,5,2019,gingerbread


In [71]:
df11 = pd.merge(df10,df6, on ='claim_id', how = 'inner')

In [72]:
df11

,date,platform,searchvolume,claim_id,week_number,year_new,claim name,product_id
0,01-01-2014,google,6613,8,1,2014,low carb,26
1,01-01-2014,google,6613,8,1,2014,low carb,29
2,01-01-2014,google,6613,8,1,2014,low carb,86
3,01-01-2014,google,6613,8,1,2014,low carb,87
4,01-01-2014,google,6613,8,1,2014,low carb,120
...,...,...,...,...,...,...,...,...
100858375,29-06-2019,google,41,433,26,2019,gingerbread,18931
100858376,29-06-2019,google,41,433,26,2019,gingerbread,18932
100858377,29-06-2019,google,41,433,26,2019,gingerbread,26054
100858378,29-06-2019,google,41,433,26,2019,gingerbread,26059


In [73]:
map1 = dict(zip(df3['product_id'],df3['sales_dollars_value']))

In [74]:
map1

{1: 4579.0,
 3: 4638.0,
 4: 15.0,
 6: 14108.0,
 7: 33879.0,
 8: 98247.0,
 9: 368551.0,
 11: 9578.0,
 12: 28214.0,
 14: 17483.0,
 15: 13000.0,
 16: 48416.0,
 17: 104835.0,
 18: 125156.0,
 19: 39473.0,
 20: 241.0,
 21: 20.0,
 23: 12962.0,
 33: 134.0,
 34: 46.0,
 35: 201.0,
 36: 72.0,
 37: 69.0,
 65: 625.0,
 75: 13.0,
 90: 355.0,
 116: 3.0,
 337: 436.0,
 338: 30.0,
 342: 124.0,
 343: 17.0,
 345: 1031.0,
 346: 10691.0,
 347: 13150.0,
 348: 12851.0,
 349: 184.0,
 350: 19.0,
 352: 2198.0,
 353: 5913.0,
 354: 8952.0,
 355: 7043.0,
 356: 8552.0,
 357: 3.0,
 358: 82.0,
 362: 17.0,
 364: 4398.0,
 368: 9.0,
 369: 2711.0,
 370: 1.0,
 371: 13.0,
 372: 7.0,
 379: 1.0,
 386: 1133.0,
 387: 555.0,
 388: 2024.0,
 389: 7.0,
 390: 2.0,
 391: 1.0,
 392: 6870.0,
 393: 5.0,
 395: 9.0,
 396: 24.0,
 405: 4.0,
 414: 15.0,
 415: 14.0,
 418: 0.0,
 419: 68529.0,
 420: 52820.0,
 422: 17.0,
 423: 17.0,
 424: 2.0,
 429: 634.0,
 437: 8.0,
 448: 3.0,
 449: 15.0,
 450: 6.0,
 451: 7.0,
 483: 1209.0,
 587: 115386.0,
 589:

In [75]:
df11['sales_dollars_value'] = df11['product_id'].map(map1)

In [76]:
df11

,date,platform,searchvolume,claim_id,week_number,year_new,claim name,product_id,sales_dollars_value
0,01-01-2014,google,6613,8,1,2014,low carb,26,17.0
1,01-01-2014,google,6613,8,1,2014,low carb,29,2.0
2,01-01-2014,google,6613,8,1,2014,low carb,86,NaN
3,01-01-2014,google,6613,8,1,2014,low carb,87,NaN
4,01-01-2014,google,6613,8,1,2014,low carb,120,2.0
...,...,...,...,...,...,...,...,...,...
100858375,29-06-2019,google,41,433,26,2019,gingerbread,18931,964.0
100858376,29-06-2019,google,41,433,26,2019,gingerbread,18932,213.0
100858377,29-06-2019,google,41,433,26,2019,gingerbread,26054,12564.0
100858378,29-06-2019,google,41,433,26,2019,gingerbread,26059,26.0


In [77]:
df11 = df11[['claim name','sales_dollars_value','year_new']]

In [78]:
df12= df11[df11['year_new'] == 2017]

In [79]:
df12

,claim name,sales_dollars_value,year_new
15222480,low carb,17.0,2017
15222481,low carb,2.0,2017
15222482,low carb,NaN,2017
15222483,low carb,NaN,2017
15222484,low carb,2.0,2017
...,...,...,...
100857410,nuts,351.0,2017
100857411,nuts,1704.0,2017
100857412,nuts,8417.0,2017
100857413,nuts,275.0,2017


In [80]:
df13= df11[df11['year_new'] == 2018]

In [81]:
df13

,claim name,sales_dollars_value,year_new
27254955,low carb,17.0,2018
27254956,low carb,2.0,2018
27254957,low carb,NaN,2018
27254958,low carb,NaN,2018
27254959,low carb,2.0,2018
...,...,...,...
100856914,mackerel,721.0,2018
100856915,mackerel,180.0,2018
100856916,mackerel,NaN,2018
100856917,mackerel,328.0,2018


In [82]:
merge1 = df12.groupby(['claim name'])['sales_dollars_value'].sum().reset_index()

In [83]:
merge1.sort_values(by = 'sales_dollars_value', ascending = False, inplace = True)

In [84]:
merge1

,claim name,sales_dollars_value
15,low carb,3.989156e+10
1,blueberry,8.022512e+09
11,gmo free,7.763893e+09
9,ethnic & exotic,5.512842e+09
24,salmon,3.704787e+09
26,soy foods,2.488665e+09
16,low sodium,1.685593e+09
19,no additives/preservatives,1.293179e+09
14,low calorie,9.723005e+08
3,chicken,9.412585e+08


In [85]:
merge2 = df13.groupby(['claim name'])['sales_dollars_value'].sum().reset_index()

In [86]:
merge2

,claim name,sales_dollars_value
0,beef hamburger,3.265133e+07
1,blueberry,1.730261e+10
2,bone health,1.404313e+07
3,chicken,1.699847e+09
4,convenience - easy-to-prepare,5.885928e+06
5,crab,1.305475e+09
6,energy/alertness,1.540680e+07
7,ethical - not specific,6.684570e+05
8,ethical - packaging,1.388601e+08
9,ethnic & exotic,1.490399e+10


In [87]:
merge2.sort_values(by = 'sales_dollars_value', ascending = False, inplace = True)

In [88]:
merge2

,claim name,sales_dollars_value
15,low carb,1.546201e+11
1,blueberry,1.730261e+10
9,ethnic & exotic,1.490399e+10
23,soy foods,9.329602e+09
11,gmo free,7.411988e+09
22,salmon,6.064873e+09
3,chicken,1.699847e+09
10,french bisque,1.661183e+09
16,low sodium,1.653789e+09
5,crab,1.305475e+09


In [89]:
final_merge = pd.merge(merge1,merge2, on = 'claim name', how = 'inner')

In [90]:
final_merge

,claim name,sales_dollars_value_x,sales_dollars_value_y
0,low carb,3.989156e+10,1.546201e+11
1,blueberry,8.022512e+09,1.730261e+10
2,gmo free,7.763893e+09,7.411988e+09
3,ethnic & exotic,5.512842e+09,1.490399e+10
4,salmon,3.704787e+09,6.064873e+09
5,soy foods,2.488665e+09,9.329602e+09
6,low sodium,1.685593e+09,1.653789e+09
7,no additives/preservatives,1.293179e+09,1.141041e+09
8,low calorie,9.723005e+08,9.175230e+08
9,chicken,9.412585e+08,1.699847e+09


In [91]:
final_merge['% change'] = ((final_merge.sales_dollars_value_y - final_merge.sales_dollars_value_x)/final_merge.sales_dollars_value_x*100)

In [92]:
final_merge.sort_values(by = '% change', ascending= False,inplace = True)

In [93]:
final_merge.rename(columns = {'sales_dollars_value_x': 'sales_dollars_value_2017'}, inplace = True)

In [94]:
final_merge.rename(columns = {'sales_dollars_value_y': 'sales_dollars_value_2018'}, inplace = True)

In [95]:
final_merge

,claim name,sales_dollars_value_2017,sales_dollars_value_2018,% change
0,low carb,3.989156e+10,1.546201e+11,287.601078
5,soy foods,2.488665e+09,9.329602e+09,274.883721
12,crab,3.744003e+08,1.305475e+09,248.684211
3,ethnic & exotic,5.512842e+09,1.490399e+10,170.350404
10,french bisque,6.713494e+08,1.661183e+09,147.439353
21,bone health,6.400492e+06,1.404313e+07,119.407008
1,blueberry,8.022512e+09,1.730261e+10,115.675676
9,chicken,9.412585e+08,1.699847e+09,80.592992
4,salmon,3.704787e+09,6.064873e+09,63.703704
13,low sugar,1.639195e+08,2.564912e+08,56.473829
